In [2]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load the supply chain data
df = pd.read_csv("supply_chain_data.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

color_map = {'Category1': 'pink', 'Category2': 'violet', 'Category3': 'red'}

# layout of the app
app.layout = html.Div([
    html.H1("SUPPLY CHAIN ANALYSIS ON COSMETICS DASHBOARD", style={'textAlign': 'center', 'color': '#003366'}),
    
    # Bubble chart for Number of products sold over time
    dcc.Graph(
        id='bubble chart',
        figure=px.scatter(df, x='Product type', y='Number of products sold', size='Revenue generated',
                          color='Customer demographics', color_discrete_map=color_map,
                          title='Number of Products Sold Over Time')
    ),

    # heatmap for Customer demographics and Transportation modes distribution
    dcc.Graph(
        id='heatmap',
        figure=px.imshow(df.groupby(['Customer demographics', 'Transportation modes']).size().unstack(),
                     labels=dict(x='Transportation modes', y='Customer demographics', color='Count'),
                     title='Customer Demographics and Transportation Modes Distribution',
                     color_continuous_scale=[
                         (0, 'red'),
                         (0.5, 'blue'),
                         (1, 'pink')
                     ])
),

    # Error Bar plot for Stock levels vs. Lead times
    dcc.Graph(
        id='error bars',
        figure=px.scatter(df, x='Stock levels', y='Lead times', error_y='Order quantities',
                      title='Stock levels vs. Lead times',
                      color='Shipping carriers',  
                      color_discrete_sequence=px.colors.qualitative.Set1 
                      )
),

    # Box plot for Manufacturing costs by Shipping carriers
    dcc.Graph(
        id='box plot',
        figure=px.box(df, x='Shipping carriers', y='Manufacturing costs',
                  title='Manufacturing Costs by Shipping Carriers',
                  color='Shipping carriers',  
                  color_discrete_map={'Shipping carriers': 'purple', 
                                      'Carrier1': 'blue',
                                      'Carrier2': 'green',
                                      'Carrier3': 'orange'}
                  )
),

    # 3D plot for Defect rates
dcc.Graph(
    id='3d plot',
    figure=px.scatter_3d(df, x='Defect rates', y='Shipping carriers', z='Manufacturing costs',
                         title='Defect Rates, Shipping Carriers, and Manufacturing Costs',
                         color='Shipping carriers', 
                         color_discrete_map={'Shipping carriers': 'orange'},
                         symbol='Manufacturing costs',  
                         symbol_sequence=['circle'],  
                         symbol_map={'Manufacturing costs': 'square'},
                         opacity=0.8 
                         )
),

    # Scatter plot for Stock levels vs. Revenue generated (Interactive)
    dcc.Graph(
        id='scatter plot',
        figure={}
    ),

    # Dropdown for selecting Product types
    dcc.Dropdown(
        id='product-type dropdown',
        options=[{'label': product_type, 'value': product_type} for product_type in df['Product type'].unique()],
        multi=True,
        value=df['Product type'].unique()
    ),
])

# callback function to update Scatter plot based on selected Product types
@app.callback(
    Output('scatter plot', 'figure'),
    [Input('product-type dropdown', 'value')]
)
def update_scatter_plot(selected_product_types):
    filtered_df = df[df['Product type'].isin(selected_product_types)]
    fig = px.scatter(filtered_df, x='Stock levels', y='Revenue generated', color='Product type',
                     title='Stock levels vs. Revenue generated (Selected Product Types)',
                     labels={'Stock levels': 'Stock Levels', 'Revenue generated': 'Revenue Generated'})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)